動画ファイルの字幕データ・文字起こしを生成する（Google Colabでの実行を想定）

参考にしたサイト：<br>
https://qiita.com/vfuji/items/e836f432ed5fb75f4306 <br>
https://qiita.com/szktmyk38f/items/374f24d06fe277a1922a <br>
https://virment.com/extract-audio-data-from-video-using-ffmepg/ <br>


In [ ]:
# ffmpeg, whisperをインストール（終わるまで数分かかる）
!sudo apt update && sudo apt install ffmpeg
!pip3 install -U openai-whisper
import torch
# PyTorchがGPU対応か確認 (TrueならばOK。Falseの場合はランタイム設定がT4 GPUではないかも)
print(torch.cuda.is_available())

In [ ]:
# 音声だけを取り出す
!ffmpeg -i zoom録画.mp4  -f mp3 -ab 256k audio-ws09.mp3
# whisperでの文字起こし（20分間の音声の場合、10分間程度で出来た。）
!whisper audio-ws09.mp3 --language Japanese --device cuda --output_dir ./ --model large

In [ ]:
# Google Colab のシークレットを読み込みます
# これを実行する前に、左端の鍵アイコンをクリックし、「新しいシークレットを追加」で、
# geminikey という名前でGemini APIキーの値を保存しておいてください。
from google.colab import userdata
geminikey = userdata.get('geminikey')

In [ ]:
import google.generativeai as genai
genai.configure(api_key=geminikey)

#gemini_model = 'gemini-1.5-flash-latest'
gemini_model = 'gemini-1.5-pro-latest'

model = genai.GenerativeModel(
    gemini_model,
    generation_config=genai.types.GenerationConfig(
        temperature=1.0
    )
)
#https://ai.google.dev/pricing

In [ ]:
with open("audio-ws09.txt", "r") as file:
    lecture1 = file.read()
    print(lecture1)

In [ ]:
prompt1 = f'''
あなたはオンラインテストの開発者です。
以下の＜講義内容＞の内容を理解できているかどうかを確かめる４択問題を５問作成してください。
正解の選択肢も示してください。
各問題は、下の＜問題フォーマットの例＞のようなフォーマットで書いてください。

＜問題フォーマットの例＞
1. AI技術の基礎に対する理解とは別に、「教育・リテラシーの原則」で強調されるべきはどれですか。
A) ステークホルダーとともに国際コンセンサスを遵守すること
B) 格差と分断の防止を、関係者の利益も考慮しつつ行うこと
C) 自動化だけでなく安全性も重視した行動を心がけること
D) データの有効利用を最大限図ること
正解: B) 格差と分断の防止を、関係者の利益も考慮しつつ行うこと
解説: 教育・リテラシーの原則では、AIに関する知識と理解の平等な提供やリテラシー向上を通じて、格差と分断の予防が重要視される。

＜講義内容＞
{lecture1}
'''
messages = [
    {'role':'user', 'parts': [prompt1]}
]
response = model.generate_content(messages)
res1 = response.candidates[0].content.parts[0].text.strip()
print(res1)

In [ ]:
prompt2 = '''
さきほどの問題データを用いて、Googleフォーム上の小テストを作成する、以下のようなGASのコードを出力してください。
選択肢番号は削除してください。選択肢文字（a, b, c など）は削除してください。
各問の配点は10点にしてください。
GASコードだけを出力し、それ以外は出力しないでください。

function createForm() {
  // フォームの作成
  var form = FormApp.create('自動作成小テスト');
  form.setIsQuiz(true); // クイズ設定を有効にする

  // 問題1の追加（記述式）
  var item1 = form.addTextItem();
  item1.setTitle('日本の首都は？');
  // setCorrectAnswerは利用できない?

  // 問題2の追加（複数選択式）
  var item2 = form.addMultipleChoiceItem();
  item2.setTitle('プログラミング言語の種類として、正しいものはどれですか。')
      .setPoints(10)
      .setRequired(true)
      .setChoices([
        item2.createChoice('Python', true),
        item2.createChoice('JavaScript', true),
        item2.createChoice('Excel', false),
        item2.createChoice('Word', false)
      ]);

  const editUrl = form.getEditUrl();
  const formUrl = form.getPublishedUrl();
  Logger.log('編集URLは ' + editUrl + ' です。設定タブを開き、「成績の発表」を「送信直後」に、また、「点数」を有効にしておくと良いでしょう。');
  Logger.log('公開URLは ' + formUrl + ' です。');
}
'''

messages = [
        {"role": "user",  "parts": [prompt1]},
        {"role": "model", "parts": [res1]},
        {"role": "user",  "parts": [prompt2]},
]
response = model.generate_content(messages)
res2 = response.candidates[0].content.parts[0].text.strip()

gas_file = 'gform1.gs'
res2 = res2.replace("```javascript", '').replace("```js", '').replace("```", '')
with open(gas_file, 'w') as f:
    print(res2, file=f)

print("Googleフォームを自動作成するGASコードが生成されました。")
print(f"左端のフォルダアイコンをクリックし、生成された {gas_file} というファイルを見つけて開いてください。")
print("その内容をコピーし、別途開いた Apps Script の画面で新しいファイルに貼り付けて実行してください。")